In [2]:
from torchtext import data,datasets
from torchtext.vocab import GloVe,FastText,CharNGram
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch
from torchtext.datasets.imdb import IMDB
import sys

In [75]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [4]:
is_cuda = False

if torch.cuda.is_available():
    is_cuda=True

In [6]:
TEXT = data.Field(lower=True, batch_first=True, fix_length=200,)
LABEL = data.Field(sequential=False,)

In [8]:
train, test = IMDB.splits(TEXT, LABEL, root='.data')

KeyboardInterrupt: 

In [9]:
TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=300), max_size=10000, min_freq=10)
LABEL.build_vocab(train)

In [10]:
# 데이터셋을 배치로 나눠서 학습시킬 수 있도록 생성
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=32, device=-1,shuffle=True)
train_iter.repeat = False
test_iter.repeat = False

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [76]:
class ConvNet(nn.Module):
    
    def __init__(self,vocab,hidden_size,n_cat,bs=1,kernel_size=3,max_len=200):
        super().__init__()
        self.hidden_size = hidden_size
        self.bs = bs
        self.e = nn.Embedding(n_vocab,hidden_size)
        self.cnn = nn.Conv1d(max_len, hidden_size, kernel_size)
        self.avg = nn.AdaptiveAvgPool1d(10)
        self.fc = nn.Linear(1000,n_cat)
        self.softmax = nn.LogSoftmax(dim=-1)
        
    def forward(self,inp):
        bs = inp.size()[0]
        if bs != self.bs:
            self.bs = bs
        e_out = self.e(inp)
#         print('eout :',e_out.shape)
        cnn_o = self.cnn(e_out)
#         print('cnnout :',cnn_o.shape)
        cnn_avg = self.avg(cnn_o)
#         print('cnnavg :',cnn_avg.shape)
        cnn_avg = cnn_avg.view(self.bs,-1)
#         print('cnnavg_view :',cnn_avg.shape)
        fc = F.dropout(self.fc(cnn_avg), p=0.5)
        return self.softmax(fc)


In [16]:
n_vocab = len(TEXT.vocab)
n_hidden = 100

In [77]:
model = ConvNet(n_vocab,n_hidden,n_cat=3,bs=32,kernel_size=2)
model = model.cuda()

In [81]:
optimizer = optim.Adam(model.parameters(), lr=0.003)
# lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [19]:
# 하나의 배치를 가져오고 모델에 넣어서 테스트
k = iter(train_iter)
kk = next(k)

In [20]:
kk


[torchtext.data.batch.Batch of size 32 from IMDB]
	[.text]:[torch.LongTensor of size 32x200]
	[.label]:[torch.LongTensor of size 32]

In [78]:
out = model(kk.text.cuda())
out.shape

torch.Size([32, 3])

In [79]:
def fit(epoch,model,data_loader,phase='training',volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , batch in enumerate(data_loader):
        text , target = batch.text , batch.label
        if is_cuda:
            text,target = text.cuda(),target.cuda()
        
        if phase == 'training':
            optimizer.zero_grad()
        output = model(text)
        loss = F.nll_loss(output,target)
        
        running_loss += F.nll_loss(output,target,size_average=False).data
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * running_correct.item()/len(data_loader.dataset)
    
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss,accuracy

In [80]:
# train val 학습에 대한 loss acc 저장을 위한 공간 확보
train_losses, train_accuracy = [], []
val_losses, val_accuracy = [], []

In [82]:
%%time
train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]

for epoch in range(1,15):

    epoch_loss, epoch_accuracy = fit(epoch,model,train_iter,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model,test_iter,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

training loss is  0.84 and training accuracy is 12139/25000     48.56
validation loss is  0.74 and validation accuracy is 13536/25000     54.14
training loss is  0.66 and training accuracy is 14437/25000     57.75
validation loss is   0.7 and validation accuracy is 14264/25000     57.06
training loss is  0.59 and training accuracy is 15154/25000     60.62
validation loss is  0.76 and validation accuracy is 14072/25000     56.29
training loss is  0.54 and training accuracy is 15699/25000      62.8
validation loss is  0.82 and validation accuracy is 14218/25000     56.87
training loss is   0.5 and training accuracy is 15981/25000     63.92
validation loss is  0.94 and validation accuracy is 14195/25000     56.78
training loss is  0.46 and training accuracy is 16331/25000     65.32
validation loss is   1.1 and validation accuracy is 13741/25000     54.96
training loss is  0.42 and training accuracy is 16476/25000      65.9
validation loss is   1.3 and validation accuracy is 14233/25000   